In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize  
from sklearn.feature_extraction.text import TfidfVectorizer ,TfidfTransformer,CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, KFold
import nltk
import math

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\SAITEJA-
[nltk_data]     WORKMACHINE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SAITEJA-
[nltk_data]     WORKMACHINE\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\SAITEJA-
[nltk_data]     WORKMACHINE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv("ios.txt")

In [4]:
df["content"] = df["title"] +" " + df["text"]


In [5]:
del df['appid']
del df['id']
del df['text']
del df['title']
del df['url']
del df['userUrl']
del df["userName"]
del df["version"]


In [6]:
df

,score,content
0,1,Absolutely terrible I am literally writing thi...
1,2,"Give the people what they want Soooo, no matte..."
2,1,"Please fix... Well did an update on Tuesday, &..."
3,1,"New Update is Trash, Useless (iOS) The new upd..."
4,3,Messenger may have a problem I don’t do review...
...,...,...
3896,5,Pink-a-licous While I love the app I do not en...
3897,3,Rating the app and clothes I’ve found the app ...
3898,3,"Issue I shop in PINK 24/7, but everytime I att..."
3899,3,Wishlist??? I am very confused on why it does ...


In [7]:
y = df["score"]
X = df.drop("score",axis =1,inplace=False)


In [8]:
X_train,X_test,y_train,y_test = train_test_split(X.index,y,test_size=0.2)
df_train=pd.concat([X,y],axis=1).iloc[X_train] # return dataframe train
df_test=pd.concat([X,y],axis=1).iloc[X_test] # return dataframe test
print(df_train.shape, df_test.shape)

(3120, 2) (781, 2)


In [9]:
df_test

,content,score
681,"My favourite app, but still has some room for ...",4
1562,Works well Updated-I have revised my review to...,5
569,Reflectly has changed my life Reflectly has ch...,5
2819,Future Rides Have clear instructions and work ...,2
2854,"The update fixed it Just so everyone knows, th...",5
...,...,...
2056,"Excellent App, but could use more Languages! I...",5
2239,Arlo review The app is adequate but not much m...,3
3682,Where do I start There are many complaints I h...,3
1769,Tricks you into subscribing to Premium I’m now...,1


In [10]:
text_train=df_train["content"].values
text_test = df_test["content"].values
y_train= df_train["score"].values
y_test = df_test["score"].values

In [11]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [12]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)

In [13]:
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

In [14]:
stemed_vec = StemmedCountVectorizer(min_df= 2,ngram_range=(1,2),tokenizer = LemmaTokenizer())

In [15]:
clf = Pipeline([("vec",stemed_vec),("tf-idf",TfidfTransformer()),
	            ("SVD",TruncatedSVD(n_components=3000)),("svr",SVR(C=10,gamma=1,verbose=1))])

In [16]:
param_grid = {'svr__C': [0.001, 0.01, 0.1, 1, 10], 'svr__gamma': [0.001, 0.01, 0.1, 1],
              'SVD__n_components':[1000,2000,3000,4000],'vec_ngram_range':[(1,1),(1,2),(1,3)],
              'tfidf__use_idf': (True, False)}

In [17]:
cv = KFold(shuffle=True)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv, verbose=3)

#### Fitting the model

In [18]:
clf.fit(text_train, y_train)

[LibSVM]

Pipeline(memory=None,
         steps=[('vec',
                 StemmedCountVectorizer(analyzer='word', binary=False,
                                        decode_error='strict',
                                        dtype=<class 'numpy.int64'>,
                                        encoding='utf-8', input='content',
                                        lowercase=True, max_df=1.0,
                                        max_features=None, min_df=2,
                                        ngram_range=(1, 2), preprocessor=None,
                                        stop_words=None, strip_accents=None,
                                        token_pattern='(?u)\\b\\w\\w+\\b',
                                        tokenizer=<__main__.Lem...
                                        vocabulary=None)),
                ('tf-idf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('SVD',

#### Prediction

In [19]:
y_pred = clf.predict(text_test)
#print(y_pred)

In [59]:
#Adjust the value to 1~5
for i in range(len(y_pred)):
	if (y_pred[i]>5) : y_pred[i]=5
	if (y_pred[i]<1) : y_pred[i]=1

In [67]:
y_testreal = df_test["score"].values
acc=0
for i in range(len(y_testreal)):
    if y_test[i]== math.ceil(y_pred[i]):
        acc=acc+1
    if i <18:
        print(y_test[i], round(y_pred[i]))
acc = acc/len(y_test)
print(acc)

2 3.0
1 1.0
5 4.0
2 1.0
4 3.0
3 2.0
5 3.0
5 3.0
4 3.0
2 2.0
5 3.0
5 3.0
1 2.0
5 4.0
5 3.0
3 4.0
4 5.0
3 1.0
0.4148527528809219


In [20]:
sample = ["This app has so many updates constantly and you cant even figure out how to make the camera on this app work!! And no it's not my phone, when I go into my camera everything is crystal clear but the minute I go on your app to take a picture it's nothing but blur even when you click on what you want it to focus on... Fix the app with your next update instead of making it worst","I love this app! Its a great way to communicate with friends! I love the amazing filters, and the best part is, when you post on your story its only available to friends so any person cant see it. It is actually also a bit different from having just a private account. To be honest if i could i would rate this 1000000/10","I wanted to talk about a problem.The games in this app are always unfair!!!!Players go against the rules and some of my actions don't even count!!! I'm writing this review because of my own experience and to many others.I personally play the Snapchat Games a lot and I'm not satisfayed with the quality too,only beacuse I have a Samsung???Above all of this I'm satisfayed with the other services,I hope Team Snapchat does something for this,THANK YOU","I love Snapchat. It is a very good texting app that allows many good features. But there is one feature that is needed. Lets say someone is using your Snapchat and tries to go through your private conversations. You should be able to lock conversations so nobody can see. Like...maybe you can enter a certain code to the locked conversation? Just a recommendation. Thanks for your time.","This app is terrible. When ever I check someone's story I have to go out of the app, clear the cache&data which isn't worth it. I have also deleted the app multiple times and then it only works okay for 2 days. Posting on my story and sending and receiving snaps works okay. But when i send or recive a snap with audio it takes a long time to load, MORE THAN 5 MINUTES! And it takes even longer to view audio snaps on others stories. This is the only way to talk to some of my friends. PLEASE FIX!","I am always using the app and it is amazing. I can facetime and ring people at the same time and u cant do that on a regular phone call. I wish i could give it more than 5 stars. I recomend this app for when u want to text people but u have no data or wifi. I am always adding to my story and the best bit is that u can make a group of loads of people and u can name a group or a person. 5 Stars.","It's a great app, however I've recently had a problem where sending anything is taking longer than it should to send. & today i woke up to it not sending anything at all. updated it thinking that'll solve the issue, it has not. it also says i have no internet connection which is wrong & it's definitely not my internet. please fix this"," Please fix this app! I have to constantly close and restart the app just to get functions to act like they're supposed to. The app also makes it difficult to view my own story; typically shows a black screen instead of the video or pic I posted. 4/26/2020 update: I have been experiencing technical issues and glitches; submitted bug report and trouble ticket with no luck. PLEASE FIX BUGS AND ISSUES!","I have been using this app from past 2 years I had not faced any problems until few days ago I guess it was because of the new update .. I'm not able to send any messages to anyone all of a sudden.I hope the TeamSnap chat will fix this asap . I use Samsung galaxyA6","It is a really fun way to get photos taken on, snap/snap call friends and family and a great way to meet new people. But I wasn't happy among the amount of inappropriate content that I got when I first joined in the subscriptions page and the people who add you and just want inappropriate things, also it is like insta gram how you can get the tick that you are celebrity which I don't like. But in a good way incompletely free and easy to use"]
samplerealratings= [1,5,3,4,2,5,1,2,3,4]

In [21]:
 samplepredictions= (clf.predict(sample))

In [22]:
for i in range(len(samplepredictions)):
    if (samplepredictions[i]>5) : samplepredictions[i]=5
    if (samplepredictions[i]<1) : samplepredictions[i]=1
        
samplepredictions

array([1.29223095, 5.        , 3.66516441, 4.18042029, 1.91766048,
       4.88922227, 2.72093886, 1.1390284 , 2.26582551, 4.62813211])